In [1]:
import sys
import torch
import torchvision
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
from PIL import Image
import torchvision.transforms as transforms
import os
import cv2
from torch.utils.data import DataLoader, TensorDataset, Dataset
from torchvision.datasets import ImageFolder
import shutil
import random

%matplotlib inline

In [2]:
img_dir = r'../PapSmearData/smear2005/New database pictures'
# dirs = os.listdir(train_dir)
# os.mkdir(r'../PapSmearData/smear2005/test')
# test_path = r'../PapSmearData/smear2005/test'

# for d in dirs:
#     os.mkdir(test_path +'/' + d)
#     files = os.listdir(train_dir + '/' + d)
#     idxes = np.arange(len(files))
#     test_idx = random.choice(idx, int(len(files)*0.2), replace=False)
#     for idx in test_idx:
        
#         a_path = train_dir + '/' + d + '/' + f
#         b_path = test_path +'/' + d
#         shutil.move(a_path, b_path)

In [3]:
transformer = transforms.Compose([transforms.Resize((224,224)),
                                 transforms.RandomRotation((-180, 180)),
                                 transforms.ToTensor()]
                                )
def default_loader(path):
    img = Image.open(path).convert('RGB')
#     print(type(img))
#     img = np.transpose(img, (2,0,1))
    return img
mydataset = ImageFolder(img_dir, transform = transformer, loader = default_loader)

In [4]:
train_loader = DataLoader(dataset = mydataset, batch_size=32, shuffle = True)

In [5]:
EPOCH = 100
CLASSES = 7
LR = 0.0001
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [6]:
from torchvision.models import resnet18
model = resnet18(pretrained=True)
for param in model.parameters():
    param.requires_grad = False

fc_features = model.fc.in_features
model.fc = torch.nn.Linear(fc_features, 7)

optimizer = optim.SGD(model.fc.parameters(), lr=LR, momentum=0.9)
criterion = nn.CrossEntropyLoss()

model.to(DEVICE)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Co

In [7]:
def train(model, device, train_loader, optimizer, epoch):
    print(device)
    model.train()
    for e in range(1, epoch+1):
        for batch_idx, (data, target) in enumerate(train_loader):
            data, target = data.to(device), target.to(device)
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
        
        print ('Train Epoch: {}\t Loss: {:.6f}\n'.format(e,loss.item()))

In [8]:
train(model, DEVICE, train_loader, optimizer, EPOCH)

cuda
Train Epoch: 1	 Loss: 2.038768

Train Epoch: 2	 Loss: 1.791988

Train Epoch: 3	 Loss: 1.889584

Train Epoch: 4	 Loss: 1.743307

Train Epoch: 5	 Loss: 1.623642

Train Epoch: 6	 Loss: 1.869009

Train Epoch: 7	 Loss: 1.629328

Train Epoch: 8	 Loss: 1.559858

Train Epoch: 9	 Loss: 1.651184

Train Epoch: 10	 Loss: 1.506958

Train Epoch: 11	 Loss: 1.710099

Train Epoch: 12	 Loss: 1.574031

Train Epoch: 13	 Loss: 1.589331

Train Epoch: 14	 Loss: 1.289456

Train Epoch: 15	 Loss: 1.571893

Train Epoch: 16	 Loss: 1.434971

Train Epoch: 17	 Loss: 1.301118

Train Epoch: 18	 Loss: 1.433402

Train Epoch: 19	 Loss: 1.625088

Train Epoch: 20	 Loss: 1.463424

Train Epoch: 21	 Loss: 1.256392

Train Epoch: 22	 Loss: 1.763113

Train Epoch: 23	 Loss: 1.274398

Train Epoch: 24	 Loss: 1.338274

Train Epoch: 25	 Loss: 1.353962

Train Epoch: 26	 Loss: 1.515621

Train Epoch: 27	 Loss: 1.458176

Train Epoch: 28	 Loss: 1.369100

Train Epoch: 29	 Loss: 1.158111

Train Epoch: 30	 Loss: 1.372669

Train Epoch: 3

In [11]:
torch.cuda.is_available()

True

In [11]:
mydataset.class_to_idx

{'.ipynb_checkpoints': 0,
 'carcinoma_in_situ': 1,
 'light_dysplastic': 2,
 'moderate_dysplastic': 3,
 'normal_columnar': 4,
 'normal_intermediate': 5,
 'normal_superficiel': 6,
 'severe_dysplastic': 7}